# sample generator for embedded questions
This notebook is for creating the random sample .txt files for the experiment.

In [1]:
import pandas as pd
import numpy as np
import re

 # Contents
 1. [Constrain the data set to stimuli set](#Constrain-the-dataset-to-stimuli-set)
 2. [Figuring out the distribution of factors per list](#Figuring-out-the-distribution-of-factors-per-list)
 3. [Figure out how to collapse the matrix verb columns](#Figure-out-how-to-collapse-the-matrix-verb-columns)
 4. [Add in the paraphrases](#Add-in-the-paraphrases)
 5. [Split EntireSentence on Question](#Split-EntireSentence-on-Question)
     2. [Wh Balancing](#Wh-Balancing)
         1. [Who](#Who)
         2. [What](#What)
         3. [Where](#Where)
         4. [When](#When)
         5. [How](#How)
 8. [Generating-random-samples](#Generating-random-samples)
     1. [First Iteration](#First-Iteration)
     2. [Second Iteration](#Second-Itreation)
     3. [Third Iteration](#Third-Iteration)
     4. [Fourth Iteration](#Fourth-Iteration)
     5. [Fifth Iteration](#Fifth-Iteration)
     6. [Sixth Iteration](#Sixth-Iteration)
     7. [Final Set](#Final-Set)

In [2]:
# import the database file from the TGrep2 searching
df = pd.read_csv("../results/swbd.tab", sep='\t', engine='python')

In [3]:
df.head()

,Item_ID,Sentence,HaveNeedTo,Finite,ModalPresent,QuestionType,EmbeddedSQ,DegreeQ,SubjectAuxInv,WhAll,...,FullWhPhrase,JustMatrixClause,DeterminerSubjPresent,DeterminerNonSubjPresent,WhNode,WhParse,Question,SentenceParse,WhPhaseType,IdentityQ
0,3:43,"uh, first, um, i need *-1 to know, uh, how do ...",no,yes,no,embedded,yes,no,yes,NaN,...,NaN,"*-1 to know, uh, how do you feel *t*-2 about, ...",no,no,NaN,(WRB how),"how do you feel *t*-2 about, uh, about * sendi...","(TOP (S (INTJ (UH Uh)) (, ,) (ADVP-TMP (RB fir...",monomorphemic,NaN
1,17:77,"and, uh, we were, i was fortunate in that i wa...",no,yes,no,subject,NaN,no,NaN,NaN,...,who,NaN,no,no,NaN,(WP who),"who, uh, *t*-1 ran the nursing home in our lit...","(TOP (S (CC And) (, ,) (INTJ (UH uh)) (, ,) (E...",monomorphemic,no
2,21:45,"so, i was very comfortable, you know, in *-1 d...",yes,yes,no,embadjunct,NaN,no,NaN,NaN,...,when,*-1 doing it when it got to the point that we ...,no,no,NaN,(WRB when),when it got to the point that we had *-2 to do...,"(TOP (S (RB So) (, ,) (NP-SBJ-1 (PRP I)) (VP (...",monomorphemic,no
3,23:31,"well, i had an occasion for my mother-in-law w...",yes,NaN,yes,relative,NaN,no,NaN,NaN,...,who,NaN,no,no,NaN,(WP who),"who *t*-1 had fell and needed * to be, you kno...","(TOP (S (INTJ (UH well)) (, ,) (NP-SBJ (PRP I)...",monomorphemic,no
4,96:22,"i mean, for somebody who *t*-1 is, you know, f...",no,yes,no,relative,NaN,no,NaN,NaN,...,who,NaN,no,no,NaN,(WP who),"who *t*-1 is, you know, for most of their life...",(TOP (FRAG (PRN (S (NP-SBJ (PRP I)) (VP (VBP m...,monomorphemic,no


In [4]:
# This makes the display show more info
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [5]:
df.pivot_table(index=['QuestionType'], values="Question", aggfunc=len).groupby(["QuestionType"]).Question.transform(lambda x: x/len(df)).reset_index()

,QuestionType,Question
0,adjunct,0.075203
1,cleft,0.064712
2,embadjunct,0.236886
3,embedded,0.160800
4,fragment,0.067654
5,relative,0.133150
6,root,0.124424
7,subject,0.137072


# Constrain the dataset to stimuli set
for experimental mock-up

First we have to remove the questions that we don;t want to include:
1. embedded questions only
2. no degree questions
3. no identity questions
4. generally only monomorphemic wh-phrases
5. only who-, what-, where-, when-, how-, and why-questions

In [6]:
critical = df[(df['QuestionType'] == 'embedded') # only embedded questions
              & 
              (df['Finite'] == 'no' )] # infinitival clauses
            

In [132]:
len(critical)

135

# Combine contexts with constrained db

In [9]:
# read in df with contexts
cntxts = pd.read_csv("swbd_contexts.csv")

In [10]:
cntxts = cntxts.drop(columns="FollowingContext")

In [11]:
# get the indixes from critical
crit_index = critical.Item_ID

### Merge back in Wh colum
so we can sample proportionately based on Wh

In [133]:
df_Wh = critical[["Item_ID","Wh","Question","MatrixPredVerb","MatrixNegPresent"]].rename(columns={"Item_ID": "TGrepID"})
df_Wh

,TGrepID,Wh,Question,MatrixPredVerb,MatrixNegPresent
58,1049:139,how,how *-5 to behave *t*-6 such that there isn't *?*,know,yes
69,1130:56,how,how *-1 to live *t*-2 very well,know,yes
267,4531:36,how,how * to combine them with other things *t*-1,wondering,NaN
379,6397:23,how,how * to compare it to other big companies *t*-1,know,yes
476,7981:19,who,who *-1 to payoff *t*-2,know,yes
491,8085:15,what,what * to do *t*-1,told,NaN
583,9927:18,how,how * to act yet *t*-1,know,yes
805,13907:24,how,how * to grow *t*-1,teaches,NaN
865,14735:59,how,how * to cope with the problem *t*-3,figure,NaN
872,14829:73,what,what * to do *t*-1 with it,know,yes


In [157]:
# subset to the items that are just the ones filtered in the previos section

# otherwise, if using the database file with contexts directly in there, then this step
# is not necessary
df_valid = cntxts[cntxts["TGrepID"].isin(set(crit_index))]

In [158]:
# Merge
df_valid = df_valid.merge(df_Wh, how = 'inner', indicator=False)
df_valid

,TGrepID,EntireSentence,PreceedingContext,Wh,Question,MatrixPredVerb,MatrixNegPresent
0,1049:139,"i also thought about it, was of, uh, *-1 waiting *-2 to talk to you that, another thing that *t*-4 occurred to me is 0 there is not so much invasion of my privacy because i know how *-5 to behave *t*-6 such that there isn't *?*.","###speakera67.###yeah.###speakerb68.###so, maybe that is a, a little bit of what privacy is *t*-1.###speakera69.###yes.###exactly.###speakerb70.###speakera71.###uh-huh.",how,how *-5 to behave *t*-6 such that there isn't *?*,know,yes
1,1130:56,"and if that's gone, um, i, i really don't know how *-1 to live *t*-2 very well,","###i actually###for al-, of the time 0 i've spent *t*-1 there, i still don't quite understand how certain things that i assume 0 *t*-2 and require privacy and require not just that you be alone but actually that you have a sense of privacy *t*-3.###speakerb94.###yes.###speakera95.###because anyone can be alone for some period of time###but for me a lot of what i do *t*-1 requires a sense that there's this invisible barrier around me which people wil-, will respect *t*-2.###speakerb96.###yes.###speakera97.",how,how *-1 to live *t*-2 very well,know,yes
2,4531:36,"and more and more people start *-2 believing them or wondering how * to combine them with other things *t*-1,","###and, part of it is 0 california, you know, in, back in the sixties, had a lot of alternative movements###speakerb98.###speakera99.###and some of them fizzled out###and some of them were disastrous###and others of them, um, had an impact on the society around here.###and one of the ones that *t*-1 had an impact was, uh, people becoming interested in alternate practices,###i'm not sure if it was a meditation practice, or if it was, you know, which *t*-1 is similar to a stress management practice or alternates to, uh, a m a approved medicine.###uh, you have, you know, major, um, acupuncture schools and things out here.### and, and you could have them around long enough",how,how * to combine them with other things *t*-1,wondering,NaN
3,6397:23,"so, i really don't know how * to compare it to other big companies *t*-1, you know.","###speakera67.###speakerb68.###that's right.###well, you know, t i, you know, t i offers some good stuff###and then i think 0 there's, i mean i think 0 there's some negatives,###but there's going *-1 to be some negatives anywhere, you know, no matter where you go *t*-2.###i have, you know, all,###this is the first really large company 0 i've worked for *t*-1.###i've always been involved in little small, you know, ind-, privately owned s-, owned firms###and so i've never had the, the big benefit package.",how,how * to compare it to other big companies *t*-1,know,yes
4,7981:19,"you don't even know who *-1 to payoff *t*-2, huh?.","###speakerb90.###yeah,###speakera91.###most of the time.###speakerb92.###but the politics, the politics gets worse in the small towns sometimes.###speakera93.###oh man, in dallas you don't even know who *t*-1's in, in administration,###there's so many of them.###speakerb94.",who,who *-1 to payoff *t*-2,know,yes
5,8085:15,* being told what * to do *t*-1 is worse.,"###speakera135.###speakerb136.###speakera137.###yeah.###no,###i know which *t*-1 is worse.###speakerb138.###yeah,###i guess so.###yeah,",what,what * to do *t*-1,told,NaN
6,9927:18,but they don't know how * to act yet *t*-1.,"###speakerb10.###yeah.###speakera11.###and the people in the city were saying, well why should i go *-1 do that *t*-2.###* make the government do that,###that's not my job.###speakerb12.###right,###they've got a lot of adjustments 0 * to make *t*-1 with *-3 coming out of what they've been through *t*-2 now,###and, uh, they've been, they've been under, under the oppression that they've been under *t*-1 for so long that now 0 they have some freedoms",how,how * to act yet *t*-1,know,yes
7,13907:24,i think 0 it teaches kids how * to grow *t*-1.,"### i, i, you know, i think that we have a bunch o

In [159]:
df_valid.head()

,TGrepID,EntireSentence,PreceedingContext,Wh,Question,MatrixPredVerb,MatrixNegPresent
0,1049:139,"i also thought about it, was of, uh, *-1 waiting *-2 to talk to you that, another thing that *t*-4 occurred to me is 0 there is not so much invasion of my privacy because i know how *-5 to behave *t*-6 such that there isn't *?*.","###speakera67.###yeah.###speakerb68.###so, maybe that is a, a little bit of what privacy is *t*-1.###speakera69.###yes.###exactly.###speakerb70.###speakera71.###uh-huh.",how,how *-5 to behave *t*-6 such that there isn't *?*,know,yes
1,1130:56,"and if that's gone, um, i, i really don't know how *-1 to live *t*-2 very well,","###i actually###for al-, of the time 0 i've spent *t*-1 there, i still don't quite understand how certain things that i assume 0 *t*-2 and require privacy and require not just that you be alone but actually that you have a sense of privacy *t*-3.###speakerb94.###yes.###speakera95.###because anyone can be alone for some period of time###but for me a lot of what i do *t*-1 requires a sense that there's this invisible barrier around me which people wil-, will respect *t*-2.###speakerb96.###yes.###speakera97.",how,how *-1 to live *t*-2 very well,know,yes
2,4531:36,"and more and more people start *-2 believing them or wondering how * to combine them with other things *t*-1,","###and, part of it is 0 california, you know, in, back in the sixties, had a lot of alternative movements###speakerb98.###speakera99.###and some of them fizzled out###and some of them were disastrous###and others of them, um, had an impact on the society around here.###and one of the ones that *t*-1 had an impact was, uh, people becoming interested in alternate practices,###i'm not sure if it was a meditation practice, or if it was, you know, which *t*-1 is similar to a stress management practice or alternates to, uh, a m a approved medicine.###uh, you have, you know, major, um, acupuncture schools and things out here.### and, and you could have them around long enough",how,how * to combine them with other things *t*-1,wondering,NaN
3,6397:23,"so, i really don't know how * to compare it to other big companies *t*-1, you know.","###speakera67.###speakerb68.###that's right.###well, you know, t i, you know, t i offers some good stuff###and then i think 0 there's, i mean i think 0 there's some negatives,###but there's going *-1 to be some negatives anywhere, you know, no matter where you go *t*-2.###i have, you know, all,###this is the first really large company 0 i've worked for *t*-1.###i've always been involved in little small, you know, ind-, privately owned s-, owned firms###and so i've never had the, the big benefit package.",how,how * to compare it to other big companies *t*-1,know,yes
4,7981:19,"you don't even know who *-1 to payoff *t*-2, huh?.","###speakerb90.###yeah,###speakera91.###most of the time.###speakerb92.###but the politics, the politics gets worse in the small towns sometimes.###speakera93.###oh man, in dallas you don't even know who *t*-1's in, in administration,###there's so many of them.###speakerb94.",who,who *-1 to payoff *t*-2,know,yes


# Split EntireSentence on Question 
This is necessary because we need to bold the question only and not the Matrix in the experimental file

In [160]:
# split EntireSentence at the string that matches the value in the 'question' column
# df_valid["Matrix"] = df_valid.apply(lambda x: x['EntireSentence'].replace(x['Question'],"<b>" + x['Question'] + "<\b>").strip(),axis=1)

In [161]:
# split that last punctuation off, to be added back on in .js script
# df_valid["punct"] = df_valid["Matrix"].apply(lambda x: list(x)[-1])

In [162]:
# remove that final punct from the Matrix column
# df_valid["Matrix"] = df_valid["Matrix"].apply(lambda x: x.replace(list(x)[-1], ' '))

In [163]:
df_valid.head()

,TGrepID,EntireSentence,PreceedingContext,Wh,Question,MatrixPredVerb,MatrixNegPresent
0,1049:139,"i also thought about it, was of, uh, *-1 waiting *-2 to talk to you that, another thing that *t*-4 occurred to me is 0 there is not so much invasion of my privacy because i know how *-5 to behave *t*-6 such that there isn't *?*.","###speakera67.###yeah.###speakerb68.###so, maybe that is a, a little bit of what privacy is *t*-1.###speakera69.###yes.###exactly.###speakerb70.###speakera71.###uh-huh.",how,how *-5 to behave *t*-6 such that there isn't *?*,know,yes
1,1130:56,"and if that's gone, um, i, i really don't know how *-1 to live *t*-2 very well,","###i actually###for al-, of the time 0 i've spent *t*-1 there, i still don't quite understand how certain things that i assume 0 *t*-2 and require privacy and require not just that you be alone but actually that you have a sense of privacy *t*-3.###speakerb94.###yes.###speakera95.###because anyone can be alone for some period of time###but for me a lot of what i do *t*-1 requires a sense that there's this invisible barrier around me which people wil-, will respect *t*-2.###speakerb96.###yes.###speakera97.",how,how *-1 to live *t*-2 very well,know,yes
2,4531:36,"and more and more people start *-2 believing them or wondering how * to combine them with other things *t*-1,","###and, part of it is 0 california, you know, in, back in the sixties, had a lot of alternative movements###speakerb98.###speakera99.###and some of them fizzled out###and some of them were disastrous###and others of them, um, had an impact on the society around here.###and one of the ones that *t*-1 had an impact was, uh, people becoming interested in alternate practices,###i'm not sure if it was a meditation practice, or if it was, you know, which *t*-1 is similar to a stress management practice or alternates to, uh, a m a approved medicine.###uh, you have, you know, major, um, acupuncture schools and things out here.### and, and you could have them around long enough",how,how * to combine them with other things *t*-1,wondering,NaN
3,6397:23,"so, i really don't know how * to compare it to other big companies *t*-1, you know.","###speakera67.###speakerb68.###that's right.###well, you know, t i, you know, t i offers some good stuff###and then i think 0 there's, i mean i think 0 there's some negatives,###but there's going *-1 to be some negatives anywhere, you know, no matter where you go *t*-2.###i have, you know, all,###this is the first really large company 0 i've worked for *t*-1.###i've always been involved in little small, you know, ind-, privately owned s-, owned firms###and so i've never had the, the big benefit package.",how,how * to compare it to other big companies *t*-1,know,yes
4,7981:19,"you don't even know who *-1 to payoff *t*-2, huh?.","###speakerb90.###yeah,###speakera91.###most of the time.###speakerb92.###but the politics, the politics gets worse in the small towns sometimes.###speakera93.###oh man, in dallas you don't even know who *t*-1's in, in administration,###there's so many of them.###speakerb94.",who,who *-1 to payoff *t*-2,know,yes


# Generating random samples

## First Iteration

6 lists of 20, 1 of 15

15 how / 6 list
4 whats / 6 lists
then random sample 1 item from the remainders

last list is whatever's left over (total 15 items)

In [164]:
#for n in range(1,7):
#     how_sample = df_valid[df_valid["Wh"] == "how"].sample(15)
#     df_valid = df_valid.drop(how_sample.index)    
    
#     what_sample = df_valid[df_valid["Wh"] == "what"].sample(4)
#     df_valid = df_valid.drop(what_sample.index)
    
#     final_sample = df_valid[
#         (df_valid["Wh"] != "how") &
#         (df_valid["Wh"] != "what")
#     ].sample(1)
#     df_valid = df_valid.drop(final_sample.index)

    
    #total = pd.concat([how_sample,what_sample,final_sample])
    # save to file
    #filename = f"../../experiments/clean_corpus/01_experiment/corpus_{n}.txt".format(n=n)
    #total.to_csv(filename,header=True,sep="\t",index=False)
    
    
    
    

## Final Set

List = 15

10 how

In [165]:
len(df_valid)

135

In [166]:
test_str = "###speakera67.###yeah.###speakerb68.###so, maybe that is a, a little bit of what privacy is *t*-1.###speakera69.###yes.###exactly.###speakerb70.###speakera71.###uh-huh."

In [203]:
def format_context(context_str):
    replace_patterns = [
        (r"###", r" "),
        (r"0", r""),
        (r"mumblex", r""),
        (r"\*\?\*", r""),
        (r"\*ich\*", r""),
        (r"\*exp\*", r""),
        (r"\s?\*t\*-\d", r""),
        (r"\*-\d", r""),
        (r"\*", r""),
        (r"\s([,.?!\s])", r"\1"),
        (r"speaker([a-z])(\d+).", r"\nSpeaker \1: "),
        (r"\s\-\d", r"")
    ]
    for pattern, repl in replace_patterns:
        context_str = re.sub(
            pattern=pattern,
            repl=repl,
            string=context_str
        )
    return context_str.strip()

In [204]:
print(format_context(test_str))

Speaker a:  yeah. 
Speaker b:  so, maybe that is a, a little bit of what privacy is. 
Speaker a:  yes. exactly. 
Speaker b:  
Speaker a:  uh-huh.


In [205]:
for pc in df_valid["PreceedingContext"].map(format_context):
    print(pc, "\n\n")

Speaker a:  yeah. 
Speaker b:  so, maybe that is a, a little bit of what privacy is. 
Speaker a:  yes. exactly. 
Speaker b:  
Speaker a:  uh-huh. 


i actually for al-, of the time i've spent there, i still don't quite understand how certain things that i assume and require privacy and require not just that you be alone but actually that you have a sense of privacy. 
Speaker b:  yes. 
Speaker a:  because anyone can be alone for some period of time but for me a lot of what i do requires a sense that there's this invisible barrier around me which people wil-, will respect. 
Speaker b:  yes. 
Speaker a: 


and, part of it is california, you know, in, back in the sixties, had a lot of alternative movements 
Speaker b:  
Speaker a:  and some of them fizzled out and some of them were disastrous and others of them, um, had an impact on the society around here. and one of the ones that had an impact was, uh, people becoming interested in alternate practices, i'm not sure if it was a meditation

In [208]:
def format_entire_sentence(sentence):
    replace_patterns = [
        (r"0", r""),
        (r"mumblex", r""),
        (r"\*\?\*", r""),
        (r"\*exp\*", r""),
        (r"\*ich\*", r""),
        (r"\s?\*t\*-\d", r""),
        (r"\*-\d", r""),
        (r"\*", r""),
        (r"\s([,.?!\s])", r"\1"),
        (r"\s\-\d", r"")
    ]
    for pattern, repl in replace_patterns:
        sentence = re.sub(
            pattern=pattern,
            repl=repl,
            string=sentence
        )
    return sentence

In [209]:
for sentence in df_valid["EntireSentence"]:
    print(format_entire_sentence(sentence))

i also thought about it, was of, uh, waiting to talk to you that, another thing that occurred to me is there is not so much invasion of my privacy because i know how to behave such that there isn't.
and if that's gone, um, i, i really don't know how to live very well,
and more and more people start believing them or wondering how to combine them with other things,
so, i really don't know how to compare it to other big companies, you know.
you don't even know who to payoff, huh?.
 being told what to do is worse.
but they don't know how to act yet.
i think it teaches kids how to grow.
and we're just really, uh, uh, now, trying to, uh, figure out how to cope with the problem because it has grown so huge.
it's like, those guys, at one point, you know, they had so much money that they didn't know what to do with it.
and, you know, we're trying to decide what, what to, what to put on one side of the house and things like that,
nobody knows what to do.
it seems to say, i'll tax them if you ca

In [194]:
mod_context = df_valid["PreceedingContext"].map(format_context)

In [195]:
mod_sentence = df_valid["EntireSentence"].map(format_entire_sentence)

In [196]:
mod_context + mod_sentence

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              Speaker a:  yeah. \nSpeaker b:  so, maybe that is a, a little bit of what privacy is. \nSpeaker a:  yes. exactly. \nSpeaker b:  \nSpeaker a:  uh-huh.i also thought about it, was of, uh, waiting to talk to you that, another thing that occurred to me is there is not so much invasion of my privacy because i know how to behave such that there isn't.
1                                                                                                                                            

In [187]:
df_valid["FullContext"] = mod_context + mod_sentence

In [188]:
for s in df_valid.FullContext:
    print(s, '\n\n')

Speaker a:  yeah. 
Speaker b:  so, maybe that is a, a little bit of what privacy is. 
Speaker a:  yes. exactly. 
Speaker b:  
Speaker a:  uh-huh.i also thought about it, was of, uh, waiting to talk to you that, another thing that occurred to me is there is not so much invasion of my privacy because i know how to behave such that there isn't. 


i actually for al-, of the time i've spent there, i still don't quite understand how certain things that i assume and require privacy and require not just that you be alone but actually that you have a sense of privacy. 
Speaker b:  yes. 
Speaker a:  because anyone can be alone for some period of time but for me a lot of what i do requires a sense that there's this invisible barrier around me which people wil-, will respect. 
Speaker b:  yes. 
Speaker a:and if that's gone, um, i, i really don't know how to live very well, 


and, part of it is california, you know, in, back in the sixties, had a lot of alternative movements 
Speaker b:  
Speaker

In [148]:
df_valid

,TGrepID,EntireSentence,PreceedingContext,Wh,Question,MatrixPredVerb,MatrixNegPresent,FullContext
0,1049:139,"i also thought about it, was of, uh, *-1 waiting *-2 to talk to you that, another thing that *t*-4 occurred to me is 0 there is not so much invasion of my privacy because i know how *-5 to behave *t*-6 such that there isn't *?*.","###speakera67.###yeah.###speakerb68.###so, maybe that is a, a little bit of what privacy is *t*-1.###speakera69.###yes.###exactly.###speakerb70.###speakera71.###uh-huh.",how,how *-5 to behave *t*-6 such that there isn't *?*,know,yes,"Speaker a: yeah.\nSpeaker b: so, maybe that is a, a little bit of what privacy is.\nSpeaker a: yes. exactly.\nSpeaker b: \nSpeaker a: uh-huh.i also thought about it, was of, uh, waiting to talk to you that, another thing that occurred to me is there is not so much invasion of my privacy because i know how to behave such that there isn't."
1,1130:56,"and if that's gone, um, i, i really don't know how *-1 to live *t*-2 very well,","###i actually###for al-, of the time 0 i've spent *t*-1 there, i still don't quite understand how certain things that i assume 0 *t*-2 and require privacy and require not just that you be alone but actually that you have a sense of privacy *t*-3.###speakerb94.###yes.###speakera95.###because anyone can be alone for some period of time###but for me a lot of what i do *t*-1 requires a sense that there's this invisible barrier around me which people wil-, will respect *t*-2.###speakerb96.###yes.###speakera97.",how,how *-1 to live *t*-2 very well,know,yes,"Speaker b: yes.\nSpeaker a: because anyone can be alone for some period of time but for me a lot of what i do requires a sense that there's this invisible barrier around me which people wil-, will respect.\nSpeaker b: yes.\nSpeaker a: and if that's gone, um, i, i really don't know how to live very well,"
2,4531:36,"and more and more people start *-2 believing them or wondering how * to combine them with other things *t*-1,","###and, part of it is 0 california, you know, in, back in the sixties, had a lot of alternative movements###speakerb98.###speakera99.###and some of them fizzled out###and some of them were disastrous###and others of them, um, had an impact on the society around here.###and one of the ones that *t*-1 had an impact was, uh, people becoming interested in alternate practices,###i'm not sure if it was a meditation practice, or if it was, you know, which *t*-1 is similar to a stress management practice or alternates to, uh, a m a approved medicine.###uh, you have, you know, major, um, acupuncture schools and things out here.### and, and you could have them around long enough",how,how * to combine them with other things *t*-1,wondering,NaN,"Speaker b: \nSpeaker a: and some of them fizzled out and some of them were disastrous and others of them, um, had an impact on the society around here. and one of the ones that had an impact was, uh, people becoming interested in alternate practices, i'm not sure if it was a meditation practice, or if it was, you know, which is similar to a stress management practice or alternates to, uh, a m a approved medicine. uh, you have, you know, major, um, acupuncture schools and things out here. and, and you could have them around long enoughand more and more people start believing them or wondering how to combine them with other things,"
3,6397:23,"so, i really don't know how * to compare it to other big companies *t*-1, you know.","###speakera67.###speakerb68.###that's right.###well, you know, t i, you know, t i offers some good stuff###and then i think 0 there's, i mean i think 0 there's some negatives,###but there's going *-1 to be some negatives anywhere, you know, no matter where you go *t*-2.###i have, you know, all,###this is the first really large company 0 i've worked for *t*-1.###i've always been involved in little small, you know, ind-, privately owned s-, owned firms###and so i've never had the, the big benefit package.",how,how * to c

In [210]:
df_valid['PreceedingContext'] = mod_context
df_valid

,TGrepID,EntireSentence,PreceedingContext,Wh,Question,MatrixPredVerb,MatrixNegPresent,FullContext
0,1049:139,"i also thought about it, was of, uh, *-1 waiting *-2 to talk to you that, another thing that *t*-4 occurred to me is 0 there is not so much invasion of my privacy because i know how *-5 to behave *t*-6 such that there isn't *?*.","Speaker a: yeah. \nSpeaker b: so, maybe that is a, a little bit of what privacy is. \nSpeaker a: yes. exactly. \nSpeaker b: \nSpeaker a: uh-huh.",how,how *-5 to behave *t*-6 such that there isn't *?*,know,yes,"Speaker a: yeah. \nSpeaker b: so, maybe that is a, a little bit of what privacy is. \nSpeaker a: yes. exactly. \nSpeaker b: \nSpeaker a: uh-huh.i also thought about it, was of, uh, waiting to talk to you that, another thing that occurred to me is there is not so much invasion of my privacy because i know how to behave such that there isn't."
1,1130:56,"and if that's gone, um, i, i really don't know how *-1 to live *t*-2 very well,","i actually for al-, of the time i've spent there, i still don't quite understand how certain things that i assume and require privacy and require not just that you be alone but actually that you have a sense of privacy. \nSpeaker b: yes. \nSpeaker a: because anyone can be alone for some period of time but for me a lot of what i do requires a sense that there's this invisible barrier around me which people wil-, will respect. \nSpeaker b: yes. \nSpeaker a:",how,how *-1 to live *t*-2 very well,know,yes,"i actually for al-, of the time i've spent there, i still don't quite understand how certain things that i assume and require privacy and require not just that you be alone but actually that you have a sense of privacy. \nSpeaker b: yes. \nSpeaker a: because anyone can be alone for some period of time but for me a lot of what i do requires a sense that there's this invisible barrier around me which people wil-, will respect. \nSpeaker b: yes. \nSpeaker a:and if that's gone, um, i, i really don't know how to live very well,"
2,4531:36,"and more and more people start *-2 believing them or wondering how * to combine them with other things *t*-1,","and, part of it is california, you know, in, back in the sixties, had a lot of alternative movements \nSpeaker b: \nSpeaker a: and some of them fizzled out and some of them were disastrous and others of them, um, had an impact on the society around here. and one of the ones that had an impact was, uh, people becoming interested in alternate practices, i'm not sure if it was a meditation practice, or if it was, you know, which is similar to a stress management practice or alternates to, uh, a m a approved medicine. uh, you have, you know, major, um, acupuncture schools and things out here. and, and you could have them around long enough",how,how * to combine them with other things *t*-1,wondering,NaN,"and, part of it is california, you know, in, back in the sixties, had a lot of alternative movements \nSpeaker b: \nSpeaker a: and some of them fizzled out and some of them were disastrous and others of them, um, had an impact on the society around here. and one of the ones that had an impact was, uh, people becoming interested in alternate practices, i'm not sure if it was a meditation practice, or if it was, you know, which is similar to a stress management practice or alternates to, uh, a m a approved medicine. uh, you have, you know, major, um, acupuncture schools and things out here. and, and you could have them around long enoughand more and more people start believing them or wondering how to combine them with other things,"
3,6397:23,"so, i really don't know how * to compare it to other big companies *t*-1, you know.","Speaker a: \nSpeaker b: that's right. well, you know, t i, you know, t i offers some good stuff and then i think there's, i mean i think there's some negatives, but there's going to be some negatives anywhere, you know, no matter where you go. i have, you know, all, this is the first really large compa

In [211]:
df_valid['EntireSentence'] = mod_sentence
df_valid

,TGrepID,EntireSentence,PreceedingContext,Wh,Question,MatrixPredVerb,MatrixNegPresent,FullContext
0,1049:139,"i also thought about it, was of, uh, waiting to talk to you that, another thing that occurred to me is there is not so much invasion of my privacy because i know how to behave such that there isn't.","Speaker a: yeah. \nSpeaker b: so, maybe that is a, a little bit of what privacy is. \nSpeaker a: yes. exactly. \nSpeaker b: \nSpeaker a: uh-huh.",how,how *-5 to behave *t*-6 such that there isn't *?*,know,yes,"Speaker a: yeah. \nSpeaker b: so, maybe that is a, a little bit of what privacy is. \nSpeaker a: yes. exactly. \nSpeaker b: \nSpeaker a: uh-huh.i also thought about it, was of, uh, waiting to talk to you that, another thing that occurred to me is there is not so much invasion of my privacy because i know how to behave such that there isn't."
1,1130:56,"and if that's gone, um, i, i really don't know how to live very well,","i actually for al-, of the time i've spent there, i still don't quite understand how certain things that i assume and require privacy and require not just that you be alone but actually that you have a sense of privacy. \nSpeaker b: yes. \nSpeaker a: because anyone can be alone for some period of time but for me a lot of what i do requires a sense that there's this invisible barrier around me which people wil-, will respect. \nSpeaker b: yes. \nSpeaker a:",how,how *-1 to live *t*-2 very well,know,yes,"i actually for al-, of the time i've spent there, i still don't quite understand how certain things that i assume and require privacy and require not just that you be alone but actually that you have a sense of privacy. \nSpeaker b: yes. \nSpeaker a: because anyone can be alone for some period of time but for me a lot of what i do requires a sense that there's this invisible barrier around me which people wil-, will respect. \nSpeaker b: yes. \nSpeaker a:and if that's gone, um, i, i really don't know how to live very well,"
2,4531:36,"and more and more people start believing them or wondering how to combine them with other things,","and, part of it is california, you know, in, back in the sixties, had a lot of alternative movements \nSpeaker b: \nSpeaker a: and some of them fizzled out and some of them were disastrous and others of them, um, had an impact on the society around here. and one of the ones that had an impact was, uh, people becoming interested in alternate practices, i'm not sure if it was a meditation practice, or if it was, you know, which is similar to a stress management practice or alternates to, uh, a m a approved medicine. uh, you have, you know, major, um, acupuncture schools and things out here. and, and you could have them around long enough",how,how * to combine them with other things *t*-1,wondering,NaN,"and, part of it is california, you know, in, back in the sixties, had a lot of alternative movements \nSpeaker b: \nSpeaker a: and some of them fizzled out and some of them were disastrous and others of them, um, had an impact on the society around here. and one of the ones that had an impact was, uh, people becoming interested in alternate practices, i'm not sure if it was a meditation practice, or if it was, you know, which is similar to a stress management practice or alternates to, uh, a m a approved medicine. uh, you have, you know, major, um, acupuncture schools and things out here. and, and you could have them around long enoughand more and more people start believing them or wondering how to combine them with other things,"
3,6397:23,"so, i really don't know how to compare it to other big companies, you know.","Speaker a: \nSpeaker b: that's right. well, you know, t i, you know, t i offers some good stuff and then i think there's, i mean i think there's some negatives, but there's going to be some negatives anywhere, you know, no matter where you go. i have, you know, all, this is the first really large company i've worked for. i've always been involved in little smal

In [212]:
df_valid.to_csv("test_items_with_FullContext")

In [178]:
s = "###and, you know, they can get a, they can grasp the points.###can they convey the data verbally or in writing.###and that's what *t*-1's, you know, really scary to me.###uh, i would really, you know,###there's such a, a push among young mothers these days *-1 to make * sure 0 their child is computer literate.###i would really think that they should be stressing more can the kid write a thought and at an early age.###and if they can't *?*, i mean if they have missed that training, then somebody, you know, before you're, you're start *-4 penalizing them with bad grades for *-2 not being able *-3 to communicate what they're thinking *t*-1, teach them these basic skills.###speakerb77.###yeah.###it *exp*-1's pretty sad * to think, uh, about those who, even today, *t*-2 are graduating from school"
print(format_context(s))

###    and, you know, they can get a, they can grasp the points. can they convey the data verbally or in writing. and that's what *t*-1's, you know, really scary to me. uh, i would really, you know, there's such a, a push among young mothers these days *-1 to make * sure 0 their child is computer literate. i would really think that they should be stressing more can the kid write a thought and at an early age. and if they can't *?*, i mean if they have missed that training, then somebody, you know, before you're, you're start *-4 penalizing them with bad grades for *-2 not being able *-3 to communicate what they're thinking *t*-1, teach them these basic skills. speakerb77. yeah. it *exp*-1's pretty sad * to think, uh, about those who, even today, *t*-2 are graduating from school 


0   and, you know, they can get a, they can grasp the points. can they convey the data verbally or in writing. and that's what *t*-1's, you know, really scary to me. uh, i would really, you know, there's such